DEPENDENCIES

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold,GridSearchCV,cross_validate
from sklearn.metrics import accuracy_score
from scipy import stats
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

TRAIN DATA

In [21]:
train_positive = np.fromfile('exp1-train-400pos.bin', dtype=np.float32).reshape([400, 5, 768])
train_negative = np.fromfile('exp1-train-2000neg.bin', dtype=np.float32).reshape([2000, 5, 768])

train_data = np.concatenate([train_positive, train_negative])

TEST DATA

In [22]:
test_data = np.fromfile('exp12-test-1200.bin', dtype=np.float32).reshape([1200, 5, 768])

SHORTCUTS

In [23]:
#X_train = train_data
X_train = np.vstack(train_data)
X_train.reshape(2400, 5, 768)
#Y_train = labels
Y_train = np.hstack(labels)
Y_train.reshape(2400, 5)
X_test = np.vstack(test_data)
X_test.reshape(1200, 5, 768)
#X_test = np.vstack(test_data)
print(X_train.shape, Y_train.shape, X_test.shape)
print(X_train[0])

(12000, 768) (12000,) (6000, 768)
[ 1.08586235e+02  1.14794952e+02  1.23190514e+02  1.32504608e+02
  1.41515457e+02  1.49427124e+02  1.56108215e+02  1.61955841e+02
  1.67288696e+02  1.71839630e+02  1.75070023e+02  1.76774368e+02
  1.77507187e+02  1.78226578e+02  1.79401550e+02  1.80828903e+02
  1.82136230e+02  1.83077560e+02  1.83749725e+02  1.84602203e+02
  1.86079987e+02  1.88321823e+02  1.91025635e+02  1.93790787e+02
  1.96549866e+02  1.99245392e+02  2.01259644e+02  2.01539474e+02
  1.98953522e+02  1.92531311e+02  1.81848648e+02  1.66882477e+02
  1.47665588e+02  1.24511978e+02  9.83398590e+01  7.04397812e+01
  4.22396317e+01  1.56336889e+01 -6.84600735e+00 -2.24543076e+01
 -2.92619457e+01 -2.70822144e+01 -1.77711601e+01 -4.73619127e+00
  8.48418045e+00  2.00014114e+01  3.01652813e+01  4.07219467e+01
  5.33941536e+01  6.85674438e+01  8.49464264e+01  1.00290062e+02
  1.12652435e+02  1.21142487e+02  1.25731224e+02  1.26819977e+02
  1.25193588e+02  1.21995239e+02  1.18706863e+02  1.1687

MODEL TRAINING - Logistic Regression

In [7]:
model = LogisticRegression()
gkf = GroupKFold(n_splits = 5)
param_grid = {'classifier__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4, 5, 8, 10, 12, 15]}
pipe = Pipeline([('scaler', StandardScaler()), ('classifier', model)])
gscv = GridSearchCV(pipe, param_grid, cv = gkf, n_jobs = 32)
gscv.fit(X_train, Y_train, groups = groups)
#model.fit(X_train, Y_train)

/home/thi/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=GroupKFold(n_splits=5),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier', LogisticRegression())]),
             n_jobs=32,
             param_grid={'classifier__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4, 5,
                                           8, 10, 12, 15]})

MODEL TRAINING - Random Forest Classifier

In [25]:
#MODEL TRAINING FINAL

forest_FINAL = RandomForestRegressor(random_state = 0, max_features = 'auto', n_estimators = 102, min_samples_leaf=50)
forest_FINAL.fit(X_train, Y_train)

RandomForestRegressor(min_samples_leaf=50, n_estimators=102, random_state=0)

MODEL EVALUATION - TRAINING DATA 

In [12]:
#X_train_prediction = forest.predict(X_train)
#training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
#print('Logistic Regression Accuracy: ', training_data_accuracy)
#print('Decision Tree Accuracy: ', tree.score(X_train, Y_train))
print('Random Forest Classifier Accuracy: ', forest_FINAL.score(X_train, Y_train))

Random Forest Classifier Accuracy:  0.4471807511589615


PREDICTION - TEST DATA

In [13]:
Predictions = forest_FINAL.predict(X_test)

Predictions_Final = np.empty(1200, dtype = int)
i=0
l=0
while i<len(Predictions):
    j=i
    k=j+5
    while j<k:
        ok=0
        if(Predictions[j]==1):
          ok=1
          j=k
        j+=1
    if ok==0:
        Predictions_Final[l]=0
    else:
      Predictions_Final[l]=1
    l+=1
    i+=5
i=0 

CREATING A SUBMISSION FILE

In [14]:
Id = np.array(range(0, 1200))
output = pd.DataFrame({'Outcome' : Predictions_Final, 'Id' : Id.tolist()})

In [15]:
output.to_csv('sampleSubmission_F.csv', index=False)